# Traitement des données de raw_flows pour regroupement par destination et analyse des produits et tonnages expédiés

In [3]:
import pandas as pd
import pandas.core.frame as pf

import json, urllib
import plotly as py
import pandas as pd
import numpy as np
import plotly.graph_objects as go


In [23]:
# Aller chercher les données de ports via l'API (à rajouter dans le load_data.sh)

#curl -o data/navigo_ports.csv "data.portic.fr/api/ports/?format=csv&srid=4326"


ports = pd.read_csv("http://data.portic.fr/api/ports/?format=csv&srid=4326", encoding='utf8')


In [27]:
# Récupérer les raw_flows
#http://data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo

#curl -o ./raw_flows_1789.csv "data.portic.fr/api/rawflows/?format=csv&date=1789&params=departure_uhgs_id,departure_fr,destination_uhgs_id,destination_fr,destination_partner_balance_supp_1789,destination_substate_1789_fr,destination_state_1789_fr,tonnage,outdate_fixed,commodity_purpose,commodity_purpose2,commodity_purpose3,commodity_purpose4,all_cargos,nb_cargo"
raw_flows = pd.read_csv("./raw_flows_1789.csv", encoding='utf8', sep=',')


type(raw_flows) #list

#df_dix_ans_smog = pd.DataFrame(dix_ans_smog, columns=['destination', '1781', '1782', '1785', '1786', '1787', '1788', '1789', '1790', '1791', 'Facade'])
print(raw_flows.shape) #(10081, 16)

print(raw_flows.columns)

''' 'Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo' '''

(10081, 16)
Index(['Unnamed: 0', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nb_cargo'],
      dtype='object')


" 'Unnamed: 0', 'departure_uhgs_id', 'departure_fr',\n       'destination_uhgs_id', 'destination_fr',\n       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',\n       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',\n       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',\n       'commodity_purpose4', 'all_cargos', 'nb_cargo' "

    'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed',
       'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3',
       'commodity_purpose4', 'all_cargos', 'nbproduits'

In [28]:
# Filtrer

dunkerque = raw_flows.query('departure_fr=="Dunkerque"')
print(dunkerque.shape)
dunkerque = dunkerque.query('destination_partner_balance_supp_1789=="Etranger"') #Pas la France
dunkerque = dunkerque.query('destination_state_1789_fr!="Grande-Bretagne"') #Pas le smogglage vers l'angleterre
dunkerque = dunkerque.query('destination_uhgs_id!="C0000009"') #Afrique
dunkerque = dunkerque.query('destination_uhgs_id!="A0146289"') #Islande
dunkerque = dunkerque.query('destination_uhgs_id!="B0000715"')  #Terre-Neuve
dunkerque = dunkerque.query('commodity_purpose!="Lège"')  #pas à vide (lest)
dunkerque = dunkerque.query('commodity_purpose!="pêche"')  #pas pour la pêche
print(dunkerque.shape) #(193, 16)


print(dunkerque[['all_cargos']])

(2539, 16)
(193, 16)
                                             all_cargos
47    [{'quantity': None, 'quantity_u': None, 'commo...
167   [{'quantity': None, 'quantity_u': None, 'commo...
172   [{'quantity': None, 'quantity_u': None, 'commo...
201   [{'quantity': None, 'quantity_u': None, 'commo...
342   [{'quantity': None, 'quantity_u': None, 'commo...
...                                                 ...
9812  [{'quantity': None, 'quantity_u': None, 'commo...
9822  [{'quantity': None, 'quantity_u': None, 'commo...
9866  [{'quantity': None, 'quantity_u': None, 'commo...
9910  [{'quantity': None, 'quantity_u': None, 'commo...
9982  [{'quantity': None, 'quantity_u': None, 'commo...

[193 rows x 1 columns]


## Premier apercu des destinations d'export de Dunkerque vers l'étranger, hors GB, hors pêche, hors navires à vide. 

In [29]:


gdata = pf.DataFrame({'count' : dunkerque.groupby( [ "destination_state_1789_fr","destination_uhgs_id"] ).size()}).reset_index()
print(gdata)

pf.DataFrame(dunkerque.groupby([ "destination_state_1789_fr","destination_uhgs_id"])['tonnage'].agg(['sum','count'])).reset_index()


   destination_state_1789_fr destination_uhgs_id  count
0                   Autriche            A0041886     17
1                   Autriche            A0046492      1
2                   Autriche            A0049808     56
3                      Brême            A0691896      1
4                   Danemark            A0912818      1
5                   Danemark            A1964976      5
6                    Espagne            A0079352      7
7                    Espagne            B0000056      1
8      Etats-Unis d'Amérique            B2827324      1
9                   Hambourg            A0743522      3
10                  Portugal            A0360085      1
11                  Portugal            A0364551      1
12                  Portugal            A0374720      6
13           Provinces-Unies            A0618784      4
14           Provinces-Unies            A0620777     10
15           Provinces-Unies            A0622825      1
16           Provinces-Unies            A0624785

,destination_state_1789_fr,destination_uhgs_id,sum,count
0,Autriche,A0041886,516.0,17
1,Autriche,A0046492,60.0,1
2,Autriche,A0049808,3570.0,56
3,Brême,A0691896,70.0,1
4,Danemark,A0912818,35.0,1
5,Danemark,A1964976,134.0,5
6,Espagne,A0079352,943.0,7
7,Espagne,B0000056,120.0,1
8,Etats-Unis d'Amérique,B2827324,350.0,1
9,Hambourg,A0743522,435.0,3


In [114]:
import csv
import json

by_pointcalls = []


for _, row in dunkerque.iterrows():
  if row['all_cargos'] != '':
    truc = row['all_cargos']
    #print(truc)
    truc = truc.replace("'", '"').replace("None", 'null')
    #.replace("bœ\x9c\x9cuf", "boeuf")
    cargos = json.loads(truc)
    #print(type(cargos))
    for cargo in cargos:
      if  cargo['commodity_standardized_fr'] != 'Lest' : 
        by_pointcalls.append({
          "cargo": cargo['commodity_purpose'],
          "cargo_standardized_fr": cargo['commodity_standardized_fr'],
          **row
        })
  else:
    by_pointcalls.append({
      "cargo": "",
      "cargo_standardized_fr": "",
      **row
    })



In [115]:
#print(dunkerque.columns)
#print(['cargo', 'cargo_standardized_fr']+dunkerque.columns.tolist())
exports_dunkerque = pd.DataFrame(by_pointcalls, columns=['cargo', 'cargo_standardized_fr']+dunkerque.columns.tolist())

exports_dunkerque = exports_dunkerque.drop(['Unnamed: 0', 'commodity_purpose', 'commodity_purpose2', 'commodity_purpose3', 'commodity_purpose4', 'all_cargos'], axis=1)

print(exports_dunkerque.columns)
print(exports_dunkerque.shape) #(331, 12)
exports_dunkerque.to_csv (r'./exports_dunkerque.csv', index = None)


Index(['cargo', 'cargo_standardized_fr', 'departure_uhgs_id', 'departure_fr',
       'destination_uhgs_id', 'destination_fr',
       'destination_partner_balance_supp_1789', 'destination_substate_1789_fr',
       'destination_state_1789_fr', 'tonnage', 'outdate_fixed', 'nb_cargo'],
      dtype='object')
(331, 12)


In [4]:
import pandas as pd
import pandas.core.frame as pf

# Lire le fichier 
exports_dunkerque = pd.read_csv("./exports_dunkerque.csv")
exports_dunkerque['tonnage_per_product'] = exports_dunkerque['tonnage'] / exports_dunkerque['nb_cargo']

gdata = pf.DataFrame({'count' : exports_dunkerque.groupby( [ "destination_state_1789_fr","cargo_standardized_fr"] ).size()}).reset_index()
#print(gdata)

pd.DataFrame(exports_dunkerque.groupby([ "destination_state_1789_fr","cargo_standardized_fr"])['tonnage'].agg(['sum','count', 'mean'])).reset_index()

exports = pd.DataFrame(exports_dunkerque.groupby([ "destination_state_1789_fr","cargo_standardized_fr"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()
exports.set_axis(['destination_state_1789_fr', 'cargo_standardized_fr', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product'], axis=1, inplace=True)
#print(exports.columns)
#exports.index.names = ['destination_state_1789_fr', 'cargo_standardized_fr', 'nb_ships', 'sum_tonnage', 'nb_ships2', 'sum_tonnage_per_product']
exports.to_csv (r'./exports_dunkerque_par_pays_par_produits.csv', index = None)

exports['destination_state_1789_fr']



#https://stackoverflow.com/questions/30631841/pandas-how-do-i-assign-values-based-on-multiple-conditions-for-existing-columns
conditions = [
    exports['cargo_standardized_fr'].apply(str.lower).isin(['avoine','orge', 'farines', 'farine', 'lin', 'blé', 'bled']) | exports['cargo_standardized_fr'].str.contains('lin'),
    exports['cargo_standardized_fr'].eq('Café') | exports['cargo_standardized_fr'].isin(['Café', 'Sucre']),
    exports['cargo_standardized_fr'].apply(str.lower).isin(['vin', 'eau-de-vie']),
    exports['cargo_standardized_fr'].apply(str.lower).isin(['charbon', 'bois']),
    exports['cargo_standardized_fr'].apply(str.lower).isin(['coton']),
    exports['cargo_standardized_fr'].eq('Laine') ,
    exports['cargo_standardized_fr'].eq('Diverses marchandises'),
    exports['cargo_standardized_fr'].isin(['Peaux de boeuf', 'Fruits', 'Fèves']) ,
    exports['cargo_standardized_fr'].eq('Huile') ,	
]
#{"bled","avoine","orge","Avoine","farines","lin","blé"}'
choices = ['gold','coral', 'purple', 'black', 'lightblue', 'azure', 'lightgreen', 'pink', 'salmon']

exports['colors'] = np.select(conditions, choices, default='grey')

pd.DataFrame.sort_values(exports, by=['colors'], inplace=True)


codes, uniques = pd.factorize(exports['destination_state_1789_fr'])
codes_p, uniques_p = pd.factorize(exports['cargo_standardized_fr'])

print(codes)
print(uniques)

print(codes_p)
print(uniques_p)

exports['source'] =  codes_p
exports['target'] =  codes+len(uniques_p)

exports


[0 1 2 3 3 0 0 0 3 4 4 4 4 3 0 3 0 3 0 2 5 1 6 4 7 8 3 0 0 0 2 3 0 3 8 0 3]
Index(['Autriche', 'Etats-Unis d'Amérique', 'Portugal', 'Provinces-Unies',
       'Espagne', 'Toscane', 'République de Gênes', 'Danemark', 'Hambourg'],
      dtype='object')
[ 0  1  2  3  4  3  4  5  6  7  8  9  6  8 10 11 12 13 13 13 13 13 13 13
 13 13 14 15 16 17 17 17 18 18 18 19 19]
Index(['Laine', 'Charbon', 'Bois', 'Sucre', 'Café', 'Graines de lin', 'Orge',
       'Avoine', 'Blé', 'Farine', 'Sel', 'Futailles vides', 'Coton',
       'Diverses marchandises', 'Fèves', 'Fruits', 'Peaux de boeuf',
       'Eau-de-vie', 'Vin', 'Huile'],
      dtype='object')


C:\Users\cplume01\AppData\Local\Temp\ipykernel_38568\2591579990.py:13: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  exports = pd.DataFrame(exports_dunkerque.groupby([ "destination_state_1789_fr","cargo_standardized_fr"])['tonnage','tonnage_per_product'].agg(['count', 'sum'])).reset_index()


,destination_state_1789_fr,cargo_standardized_fr,nb_ships,sum_tonnage,nb_ships2,sum_tonnage_per_product,colors,source,target
7,Autriche,Laine,2,86.0,2,43.000000,azure,0,20
18,Etats-Unis d'Amérique,Charbon,1,350.0,1,175.000000,black,1,21
22,Portugal,Bois,1,240.0,1,240.000000,black,2,22
33,Provinces-Unies,Sucre,41,2642.0,41,1798.333333,coral,3,23
26,Provinces-Unies,Café,25,1713.0,25,1014.666667,coral,4,23
10,Autriche,Sucre,22,936.0,22,574.333333,coral,3,20
0,Autriche,Café,15,593.0,15,252.333333,coral,4,20
5,Autriche,Graines de lin,3,80.0,3,70.000000,gold,5,20
32,Provinces-Unies,Orge,3,180.0,3,180.000000,gold,6,23
13,Espagne,Avoine,3,390.0,3,390.000000,gold,7,24


In [6]:
#from https://chart-studio.plotly.com/~alishobeiri/1591/plotly-sankey-diagrams/#/ 
#https://medium.com/plotly/4-interactive-sankey-diagram-made-in-python-3057b9ee8616
#https://stackoverflow.com/questions/59845407/plotly-express-vs-altair-vega-lite-for-interactive-plots
# https://matplotlib.org/3.1.0/tutorials/colors/colors.html
#https://towardsdatascience.com/sankey-diagram-basics-with-pythons-plotly-7a13d557401a 
#https://chart-studio.plotly.com/~alishobeiri/1591/plotly-sankey-diagrams/#/
#https://errorsfixing.com/plotly-sankey-is-it-able-to-put-the-node-labels-to-the-right-for-last-layercolumn/ 

import json, urllib
import plotly as py
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Lire le fichier 
#exports = pd.read_csv("./exports_dunkerque_par_pays_par_produits.csv")

print(exports.shape)
print(pd.unique(exports['destination_state_1789_fr']).tolist()+pd.unique(exports['cargo_standardized_fr']).tolist())



fig =  go.Figure(data=[go.Sankey(
    node = dict(
      pad = 15,
      thickness = 30,
      #label =  exports['Node, Label'].dropna(axis=0, how='any'),
      #color = scottish_df['Color']
      line = dict(color = "black", width = 0.5),
      label = uniques_p.tolist()+uniques.tolist(),
      color = 'grey'
      #color = exports['colors'].dropna(axis=0, how='any')
    ),
    link = dict(
      source = exports['source'],
      target = exports['target'],
      value = exports['sum_tonnage_per_product'].dropna(axis=0, how='any'),
      color = exports['colors'].dropna(axis=0, how='any'),
  )
)])




fig.update_layout(title_text="Export des produits depuis Dunkerque", font_size=10, height = 1000, width=700)
fig.show()

import plotly.io as pio

pio.write_image(fig, './sankey_export_product_Dunkerque_1789.png', format='png', scale=1.5, width=700, height=1000)



(37, 9)
['Autriche', "Etats-Unis d'Amérique", 'Portugal', 'Provinces-Unies', 'Espagne', 'Toscane', 'République de Gênes', 'Danemark', 'Hambourg', 'Laine', 'Charbon', 'Bois', 'Sucre', 'Café', 'Graines de lin', 'Orge', 'Avoine', 'Blé', 'Farine', 'Sel', 'Futailles vides', 'Coton', 'Diverses marchandises', 'Fèves', 'Fruits', 'Peaux de boeuf', 'Eau-de-vie', 'Vin', 'Huile']


In [125]:
data = dict(
type='sankey',
node = dict(
  pad = 15,
  thickness = 20,
  line = dict(
    color = "black",
    width = 0.5
  ),
  label = ["A1", "A2", "B1", "B2", "C1", "C2"],

  color = ["blue", "blue", "blue", "blue", 
  "gray", "white"]),#attempt to make it less visible

link = dict(
  source = [0,1,0,2,3,3],
  target = [2,3,3,4,4,5],
  value = [8,4,2,8,4,2],
  # attempt to add labels
  label= [8,4,2,8,4,2]))


import plotly.offline as py
layout =  dict(
  title = "Basic Sankey Diagram",
  font = dict(
    size = 10
  ),
    annotations=[
        dict(
            x=0.25,
            y=0.75,
            text='8',
            showarrow=False
        ),
        dict(
            x=0.75,
            y=0.25,
            text='4',
            showarrow=False
        )
    ]
 )

fig = dict(data=[data], layout=layout)
py.iplot(fig, validate=False)

In [ ]:
#https://vega.github.io/vega/docs/projections/
#https://github.com/vega/vega-lite/issues/3488#issuecomment-386199585
#"scale": {"domain": [1, 25], "range":[0, 179], "clamp": true, "zero": false
# World 1789, center on England
year='1791'
map = alt.layer(
    # use the sphere of the Earth as the base layer
    alt.Chart({'sphere': True}).mark_geoshape(
        fill='#e6f3ff'
    ),
    # add a graticule for geographic reference lines
    alt.Chart({'graticule': True}).mark_geoshape(
        stroke='#ffffff', strokeWidth=1
    ),
    # and then the countries of the world
    alt.Chart(pays).mark_geoshape(
        fill='lightgray', stroke='white', strokeWidth=0.5
    ),
    #the ports
    alt.Chart(port_tomap).mark_circle().encode(
        longitude='x:Q',
        latitude='y:Q',
        size=alt.Size('{}:Q'.format(year), title='British smogglers in {}'.format(year), scale=alt.Scale(domain=[-1, 180], range=[10, 400], clamp=True, zero= False)), 
        color=alt.value('steelblue'),
        tooltip=['toponyme_standard_fr:N','{}:Q'.format(year)]
    )
).properties(
    width=400,
    height=600,
    title='British smogglers gone from Dunkirk, around 1789'
).project('conicEquidistant', scale=3000).configure_projection(center = [-4, 54]).configure_view(stroke=None)

map.save('test.png')
